In [1]:
import requests
import time
import pymysql
import os
from random import uniform
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def rest_get(apilink):
    print(apilink)
    resp = requests.get(apilink)
    if resp.status_code != 200:
        # This means something went wrong.
        raise Exception('GET {} {}'.format(apilink, resp.status_code))
    else:
        return resp

### API approach, has limits per day - not worth it, continue to full scraping approach

In [3]:
'''
links = ['https://github.com/poloclub/cnn-explainer']
github = 'https://api.github.com/'
gituser = 'https://github-user.now.sh/?username='
    
for l in links:
    repo_split = l.split('/')
    owner, repo = repo_split[-2], repo_split[-1]
    print('Requestiong: ',owner, repo)
    resp = rest_get(f'{github}repos/{owner}/{repo}/contributors')
    for user in resp.json():
        print(f'user "{user["login"]}" has {user["contributions"]} contributions')
        user_info = rest_get(gituser+user["login"])
        print(f"Name: {user_info['name']}, Description: {user_info['description']}")
'''
    

'\nlinks = [\'https://github.com/poloclub/cnn-explainer\']\ngithub = \'https://api.github.com/\'\ngituser = \'https://github-user.now.sh/?username=\'\n    \nfor l in links:\n    repo_split = l.split(\'/\')\n    owner, repo = repo_split[-2], repo_split[-1]\n    print(\'Requestiong: \',owner, repo)\n    resp = rest_get(f\'{github}repos/{owner}/{repo}/contributors\')\n    for user in resp.json():\n        print(f\'user "{user["login"]}" has {user["contributions"]} contributions\')\n        user_info = rest_get(gituser+user["login"])\n        print(f"Name: {user_info[\'name\']}, Description: {user_info[\'description\']}")\n'

### Saving author contributions to txt file

In [4]:
conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')
cursor=conn.cursor()

In [18]:

repos = []
cursor.execute("SELECT DISTINCT krw.repo.repo_url FROM krw.paper \
INNER JOIN krw.repo ON krw.repo.paper_url = krw.paper.paper_url \
WHERE krw.paper.makg_sameAs <> 'notfound'")
for r in cursor:
    repos.append(list(r)[0])
'''
cursor.execute("SELECT krw.repo.paper_url, repo_url, author \
               FROM krw.repo INNER JOIN krw.paper_author \
               ON krw.repo.paper_url=krw.paper_author.paper_url limit 5000;")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    data.append(list(r))
'''

'\ncursor.execute("SELECT krw.repo.paper_url, repo_url, author                FROM krw.repo INNER JOIN krw.paper_author                ON krw.repo.paper_url=krw.paper_author.paper_url limit 5000;")#("SELECT * FROM krw.training_view limit 10;")\nfor r in cursor:\n    data.append(list(r))\n'

In [7]:
import xlrd

repos=[]
workbook = xlrd.open_workbook("vu_uva_nl.xlsx","rb")
sheets = workbook.sheet_names()
required_data = []
sh = workbook.sheet_by_name("Sheet1")
for rownum in range(sh.nrows):
    row_valaues = sh.row_values(rownum)
    required_data.append((row_valaues[0], row_valaues[1]))
    if len(row_valaues[4])>5:
        repos.append(row_valaues[4])

In [6]:
required_data

[('VU', 'name'),
 ('A105917', 'Oana Inel'),
 ('A41114', 'Evert Haasdijk'),
 ('A61903', 'Jan Wielemaker'),
 ('A23211', 'Chantal van So'),
 ('A120874', 'Roser Morante'),
 ('A38972', 'Emiel van Miltenburg'),
 ('A83786', 'Lora Aroyo'),
 ('A111901', 'Piek Vossen'),
 ('UvA', ''),
 ('A150315', 'Jorn-Henrik Jacobsen'),
 ('A164092', 'Zhongyu Lou'),
 ('A38970', 'Emiel Hoogeboom'),
 ('A70344', 'Jorn W. T. Peters'),
 ('A93118', 'Maurice Weiler'),
 ('A137551', 'Taco Cohen'),
 ('A14026', 'Arnold W. M. Smeulders'),
 ('A61927', 'Jan van Gemert'),
 ('A41062', 'Evangelos Kanoulas'),
 ('A93368', 'Max Welling'),
 ('nl', ''),
 ('A40201', 'Erik J Bekkers'),
 ('A88041', 'Marcel A. J. van Gerven'),
 ('A43837', 'Florian Krebs'),
 ('A68383', 'Johannes Bjerva')]

In [9]:
repos

['https://github.com/ChantalvanSon/CorpusComparison',
 'https://github.com/lfsimoes/beam_paco__gtoc5',
 'https://github.com/SWI-Prolog/swish',
 'https://github.com/ChantalvanSon/CorpusComparison',
 'https://github.com/ChantalvanSon/CorpusComparison',
 'https://github.com/cltl/DutchDescriptions',
 'https://github.com/ChantalvanSon/CorpusComparison',
 'https://github.com/ChantalvanSon/CorpusComparison',
 'https://github.com/jhjacobsen/RFNN',
 'https://github.com/jhjacobsen/RFNN',
 'https://github.com/ehoogeboom/hexaconv',
 'https://github.com/ehoogeboom/hexaconv',
 'https://github.com/jhjacobsen/RFNN',
 'https://github.com/ehoogeboom/hexaconv',
 'https://github.com/jhjacobsen/RFNN',
 'https://github.com/jhjacobsen/RFNN',
 'https://github.com/cvangysel/cuNVSM',
 'https://github.com/mariogeiger/se3cnn',
 'https://github.com/tueimage/se2cnn',
 'https://github.com/yagguc/deep_impression',
 'https://github.com/jerryspan/FacebookR',
 'https://github.com/coastalcph/uuparser']

In [8]:
len(repos)

22

In [10]:
filename = 'scraped_data.txt'
if not os.path.exists(filename):
    file = open(filename, 'w+')
    file.close()

In [11]:
# Remove repos we already saved into file
with open(filename) as fp:
    line = fp.readline().split()
    while line:
        current_repo = line[0]
        if current_repo in repos:
            repos.remove(current_repo)
        line = fp.readline().split()
len(repos)

14

In [12]:
missed = []

In [13]:
driver = webdriver.Chrome(ChromeDriverManager().install()) # Firefox()

for repo in repos:
    print(repo) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(12+uniform(1, 3)) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'}) 
    if not len(boxes):
        print('NOT FOUND/LOADED')
        missed.append(repo)
    else:
        for user in boxes[0:15]:   # take just top 15 contributors, some repos might have 100+ contributors
            username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
            info_line = user.find_all(('span', {'class':'cmeta'}))
            try:
                scrape = rest_get(f'https://github.com/{username}')
            except:
                continue
            soup = BeautifulSoup(scrape.content, 'html.parser')
            time.sleep(uniform(1, 2)) # adding more delays to avoid being blocked by github
            name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
            bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
            print(f'name: {name}, bio: {bio}')

            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            if name=="":
                name = "NULL"
            else:
                name = name.replace(" ", "_")
            with open(filename, 'a') as file:
                file.write(f'{repo} {username} {name} {commits} {added} {deleted}\n')
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

[WDM] - Looking for [chromedriver 81.0.4044.138 linux64] driver in cache 
[WDM] - File found in cache by path [/home/kanter/.wdm/drivers/chromedriver/81.0.4044.138/linux64/chromedriver]


https://github.com/ChantalvanSon/CorpusComparison
NOT FOUND/LOADED

https://github.com/lfsimoes/beam_paco__gtoc5
https://github.com/lfsimoes
name: Luís F. Simões, bio: Data Scientist, AI researcher.
User lfsimoes: commits 30 , changed lines + 58,123 - 216

https://github.com/ChantalvanSon/CorpusComparison
https://github.com/ChantalvanSon
name: Chantal van Son, bio: 
User ChantalvanSon: commits 61 , changed lines + 2,903,537 - 2,687,268

https://github.com/ChantalvanSon/CorpusComparison
https://github.com/ChantalvanSon
name: Chantal van Son, bio: 
User ChantalvanSon: commits 61 , changed lines + 2,903,537 - 2,687,268

https://github.com/cltl/DutchDescriptions
NOT FOUND/LOADED

https://github.com/ChantalvanSon/CorpusComparison
https://github.com/ChantalvanSon
name: Chantal van Son, bio: 
User ChantalvanSon: commits 61 , changed lines + 2,903,537 - 2,687,268

https://github.com/ChantalvanSon/CorpusComparison
https://github.com/ChantalvanSon
name: Chantal van Son, bio: 
User ChantalvanSon:

In [20]:
missed

['https://github.com/cltl/DutchDescriptions',
 'https://github.com/cvangysel/cuNVSM',
 'https://github.com/coastalcph/uuparser']

In [15]:
driver = webdriver.Firefox() # 

for repo in repos[5401:6500]:
    print(repo) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(12+uniform(1, 3)) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'}) 
    if not len(boxes):
        print('NOT FOUND/LOADED')
        missed.append(repo)
    else:
        for user in boxes[0:15]:   # take just top 15 contributors, some repos might have 100+ contributors
            username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
            info_line = user.find_all(('span', {'class':'cmeta'}))
            try:
                scrape = rest_get(f'https://github.com/{username}')
            except:
                continue
            soup = BeautifulSoup(scrape.content, 'html.parser')
            time.sleep(uniform(1, 2)) # adding more delays to avoid being blocked by github
            name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
            bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
            print(f'name: {name}, bio: {bio}')

            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            if name=="":
                name = "NULL"
            else:
                name = name.replace(" ", "_")
            with open(filename, 'a') as file:
                file.write(f'{repo} {username} {name} {commits} {added} {deleted}\n')
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

https://github.com/yulequan/PU-Net
https://github.com/yulequan
name: Lequan Yu, bio: I am a postdoc at Stanford University working on AI for medical image analysis.
User yulequan: commits 37 , changed lines + 194,290 - 78

https://github.com/pinkeshbadjatiya/twitter-hatespeech
https://github.com/pinkeshbadjatiya
name: Pinkesh Badjatiya, bio: AI Research Engineer @adobe
User pinkeshbadjatiya: commits 30 , changed lines + 19,742 - 2,706
https://github.com/shashankg7
name: Shashank Gupta, bio: Data Scientist @Flipkart.
User shashankg7: commits 17 , changed lines + 1,935 - 17,210

https://github.com/Lornatang/tensorflow-vgg
NOT FOUND/LOADED

https://github.com/nandofioretto/GpuBE
NOT FOUND/LOADED

https://github.com/jomian78/2d_image_gans
https://github.com/jomian78
name: Joel Meyer, bio: Robotics @ Northwestern University
User jomian78: commits 50 , changed lines + 16,685 - 8,957

https://github.com/josedolz/LiviaNET
https://github.com/josedolz
name: Jose Dolz, bio: Assistant Professor at

name: Bogdan Didenko, bio: 
User BogdanDidenko: commits 2 , changed lines + 19 - 7
https://github.com/zhaoyongke
name: zhaoyongke, bio: 
User zhaoyongke: commits 1 , changed lines + 1 - 1
https://github.com/ywkim
name: Youngwook Kim, bio: 
User ywkim: commits 1 , changed lines + 1 - 1
https://github.com/tianxin1860
name: tianxin, bio: Focus on Machine Learning,  Natural Language Processing.
User tianxin1860: commits 1 , changed lines + 2 - 2
https://github.com/stefan-it
name: Stefan Schweter, bio: Developer at @dbmdz,
M.Sc Computational Linguistics, Researcher and former student @ The Center for Information and Language Processing (CIS), LMU Munich
User stefan-it: commits 1 , changed lines + 11 - 11
https://github.com/soloice
name: Chong Ruan, bio: NLPer. Know some machine learning and tensorflow.
User soloice: commits 1 , changed lines + 5 - 1
https://github.com/slavpetrov
name: Slav Petrov, bio: 
User slavpetrov: commits 1 , changed lines + 4 - 6
https://github.com/rodgzilla
name: Gr

NOT FOUND/LOADED

https://github.com/thaihungle/DMNC
NOT FOUND/LOADED

https://github.com/tensorlayer/SRGAN
https://github.com/zsdonghao
name: Hao, bio: /\︿╱\ _____________________________ \  0_ 0  /╱\╱ _____________________ \▁︹_/
User zsdonghao: commits 93 , changed lines + 31,895 - 31,227
https://github.com/akaraspt
name: Akara Supratak, bio: Lecturer at Mahidol University. Perviously, a PhD student at Imperial College London. Interested in deep learning and machine learning.
User akaraspt: commits 6 , changed lines + 79 - 14
https://github.com/Saren-Arterius
name: Saren Arterius, bio: OwO UwU
User Saren-Arterius: commits 1 , changed lines + 4 - 2
https://github.com/Laicheng0830
name: ChengLai, bio: Deep Learning Development in PCL
User Laicheng0830: commits 1 , changed lines + 1 - 1
https://github.com/ChrisWu1997
name: Rundi Wu, bio: Undergraduate at EECS, Peking University
User ChrisWu1997: commits 1 , changed lines + 3 - 3

https://github.com/ravidziv/IDNNs
NOT FOUND/LOADED

https

name: Haze Lee, bio: Programmer, who do hard in rehabilitation.
User Hazealign: commits 2 , changed lines + 11 - 13
https://github.com/zhangboyang
name: Zhang Boyang, bio: 
User zhangboyang: commits 1 , changed lines + 4 - 4
https://github.com/voormann
name: , bio: 
User voormann: commits 1 , changed lines + 30 - 0
https://github.com/mrkn
name: Kenta Murata, bio: A committer of Apache Arrow and CRuby.
User mrkn: commits 1 , changed lines + 1 - 1
https://github.com/kvfrans
name: Kevin Frans, bio: Undergrad at MIT. Working on automating design at Autodesk. Previously intern at OpenAI.
User kvfrans: commits 1 , changed lines + 10 - 4

https://github.com/Theano/Theano
https://github.com/nouiz
name: Frédéric Bastien, bio: 
User nouiz: commits 7,274 , changed lines + 199,190 - 124,177
https://github.com/abergeron
name: , bio: 
User abergeron: commits 1,802 , changed lines + 65,067 - 144,845
https://github.com/lamblin
name: Pascal Lamblin, bio: 
User lamblin: commits 1,643 , changed lines + 4

https://github.com/adalca
name: Adrian Dalca, bio: MIT and MGH (Harvard Medical School). Machine Learning in Medical Image Analysis.
User adalca: commits 121 , changed lines + 6,820 - 915
https://github.com/voxelmorph
name: , bio: dev team for voxelmorph - learning based image registration
User voxelmorph: commits 3 , changed lines + 1,368 - 674
https://github.com/balakg
name: , bio: 
User balakg: commits 2 , changed lines + 0 - 0
https://github.com/katiemlewis
name: Katie Lewis, bio: 
User katiemlewis: commits 1 , changed lines + 88 - 0

https://github.com/NeuronDroid/GVSS-S.A.Drone
https://github.com/HAN-ARK
name: Hanbyul Park, bio: 
User HAN-ARK: commits 16 , changed lines + 901 - 303
https://github.com/Zar-Par-Vanier2019
name: , bio: 
User Zar-Par-Vanier2019: commits 6 , changed lines + 464 - 34
https://github.com/TkaDanVanier2019
name: , bio: 
User TkaDanVanier2019: commits 4 , changed lines + 167 - 20
https://github.com/MakAinVanier2019
name: Ainafee Maknojia, bio: 
User MakAinVa

NOT FOUND/LOADED

https://github.com/MaybeShewill-CV/lanenet-lane-detection
NOT FOUND/LOADED

https://github.com/INK-USC/RENet
https://github.com/shanzhenren
name: Xiang Ren, bio: Asst. Professor, University of Southern California
User shanzhenren: commits 13 , changed lines + 2,748,153 - 14
https://github.com/changlinzhang
name: , bio: No silver bullet.
User changlinzhang: commits 1 , changed lines + 2,523 - 0

https://github.com/elias-ramzi/svhn_classifier
NOT FOUND/LOADED

https://github.com/EngSalem/TextClassification_Off_the_shelf
https://github.com/EngSalem
name: Mohamed Elaraby, bio: I am a Datascientist at Microsoft ATLC.
User EngSalem: commits 20 , changed lines + 158 - 74
https://github.com/mohamedhama
name: , bio: 
User mohamedhama: commits 2 , changed lines + 268,022 - 48

https://github.com/cs-heibao/PFPNet_Pytorch
NOT FOUND/LOADED

https://github.com/HKUST-KnowComp/R-Net
https://github.com/ericchansen
name: Eric Hansen, bio: I wish I could share the really cool stuff
User

name: Luca Antiga, bio: 
User lantiga: commits 4 , changed lines + 41 - 35
https://github.com/pythonokai
name: Jeremy, bio: bug
User pythonokai: commits 2 , changed lines + 79 - 7
https://github.com/pronot
name: , bio: 
User pronot: commits 2 , changed lines + 2 - 2
https://github.com/werdes72
name: Patryk Strugacz, bio: 
User werdes72: commits 1 , changed lines + 1 - 1
https://github.com/sumsuddin
name: Md Sumsuddin, bio: Software engineer with 4+ years experience in machine learning, deep learning, data science, back-end web development etc.
User sumsuddin: commits 1 , changed lines + 2 - 1
https://github.com/bgth
name: GurunathB, bio: 
User bgth: commits 1 , changed lines + 1 - 1
https://github.com/GlastonburyC
name: CG, bio: Senior Machine Learning Researcher @BenevolentAI. Biomedical Imaging, Genomics & Healthcare ML.
User GlastonburyC: commits 1 , changed lines + 19 - 17

https://github.com/YunzhuLi/InfoGAIL
https://github.com/YunzhuLi
name: Yunzhu Li, bio: PhD Candidate
User Yun

name: Jörg Bornschein, bio: 
User jbornschein: commits 27 , changed lines + 610 - 147
https://github.com/hantek
name: Jonathan Zhouhan LIN, bio: Facebook AI Research
User hantek: commits 27 , changed lines + 20,927 - 20,345
https://github.com/Thrandis
name: César Laurent, bio: 
User Thrandis: commits 16 , changed lines + 505 - 316
https://github.com/requires
name: requires.io, bio: 
User requires: commits 13 , changed lines + 18 - 18
https://github.com/aam-at
name: Alexander Matyasko, bio: 
User aam-at: commits 11 , changed lines + 1,079 - 610
https://github.com/maximecb
name: Maxime Chevalier-Boisvert, bio: CS PhD working in deep learning & robotics. Also interested in compilers, music, 3D graphics, electronics and DIY.
User maximecb: commits 10 , changed lines + 183 - 160
https://github.com/lamblin
name: Pascal Lamblin, bio: 
User lamblin: commits 7 , changed lines + 343 - 166
https://github.com/julianser
name: Iulian Vlad Serban, bio: 
User julianser: commits 6 , changed lines + 228

KeyboardInterrupt: 

In [15]:
len(missed)

5